**Clasificación con Scikit-learn y MLflow

In [1]:
!pip install pandas numpy mlflow scikit-learn


In [2]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score

from sklearn.datasets import load_breast_cancer

ModuleNotFoundError: No module named 'pandas'